# テレビ台本生成

このプロジェクトでは、RNNを使って独自の[Seinfeld](https://en.wikipedia.org/wiki/Seinfeld)TVスクリプトを生成します。 使用するのは、9シーズン分の台本を集めた[Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv)の一部です。 作成するニューラルネットワークは、この学習データで認識したパターンに基づいて、新しい「偽の」テレビ台本を生成します。

## データの入手

データは`./data/Seinfeld_Scripts.txt`にすでに用意されていますので、そのファイルを開いてテキストを見てみてください。
>* 最初のステップとして、このデータを読み込んで、いくつかのサンプルを見てみましょう。
* その後、新しいスクリプトを生成するために、RNNを定義して学習させるという課題があります。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/人工知能/deep-learning-v2-pytorchのコピー/project-tv-script-generation

/content/drive/MyDrive/人工知能/deep-learning-v2-pytorchのコピー/project-tv-script-generation


In [ ]:
"""
このセルでは何も変更しないでください
"""
# データのロード
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## データの探索
view_line_range`を使って、データのさまざまな部分を見てみましょう。そうすることで、これから扱うデータのイメージがつかめるでしょう。例えば、すべて小文字のテキストであることや、各ダイアログの新しい行が改行文字 `n` で区切られていることなどがわかります。

In [ ]:
view_line_range = (0, 10)

"""
この行より下のセルには何も手を加えないでください
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## 前処理機能の実装
データセットに最初にすることは、前処理です。 以下の前処理関数を実装してください。
- ルックアップテーブル
- 句読点のトークン化

### ルックアップテーブル
単語の埋め込みを行うためには、まず単語をIDに変換する必要があります。 この関数では、2つの辞書を作成します。
- vocab_to_int` という名前で、単語からidに変換する辞書。
- idから単語に変換するための辞書、ここでは`int_to_vocab`とします。

これらの辞書を以下の **タプル** `(vocab_to_int, int_to_vocab)` で返します。

In [ ]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    語彙のためのルックアップテーブルの作成
    :param text: 単語に分割されたテレビ台本のテキスト
    :return: 辞書のタプル (vocab_to_int, int_to_vocab)
    """
    # TODO: 関数を実装
    vocab_to_int = {word: idx for idx, word in enumerate(set(text))}
    id_to_vocab = {vocab_to_int[word]: word for word in set(text)}

    # return tuple
    return (vocab_to_int, id_to_vocab)


"""
これ以下のセルでは何も変更しないでください
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### 句読点のトークン化
ここでは、スペースを区切り文字としてスクリプトを単語の配列に分割します。 しかし、ピリオドや感嘆符などの句読点は、同じ単語に複数の ID を作成することができます。例えば、"bye "と "bye!"では2つの異なる単語IDが生成されます。

関数 `token_lookup` を実装して、"!" のような記号を "||Exclamation_Mark||" にトークン化するのに使われる dict を返します。 以下のシンボルのための辞書を作成します。シンボルがキーで、値がトークンです。
- ピリオド ( **.** )
- カンマ ( **,** )
- クォーテーションマーク ( **"**" )
- セミコロン ( **;** )
- エクスクラメーションマーク ( **!** )
- クエスチョンマーク ( **?** )
- 左カッコ ( **(**) )
- 右カッコ ( **)** )
- ダッシュ ( **-** )
- リターン ( **n** )

この辞書を使って、記号をトークン化し、その周りにデリミタ（スペース）を追加します。 これにより、各記号がそれぞれの単語として分離され、ニューラルネットワークが次の単語を予測しやすくなります。例えば、"dash "という値を使う代わりに、"||dash||"のようにしてみましょう。

www.DeepL.com/Translator（無料版）で翻訳しました。

In [ ]:
def token_lookup():
    """
    句読点をトークンに変換するdictを生成します。
    return:句読点をキーとし、トークンを値とするトークン化された辞書
    """
    # TODO: 関数の実装
    tokens = {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_Mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_Mark||',
        '?': '||Question_Mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parentheses||',
        '-': '||Dash||',
        '\n': '||Return||'
    }
    
    return tokens

"""
これ以下のセルでは何も変更しないでください
"""
tests.test_tokenize(token_lookup)

Tests Passed


## すべてのデータを前処理して保存する

以下のコードセルを実行すると、すべてのデータが前処理され、ファイルに保存されます。詳しくは `helpers.py` ファイルの `preprocess_and_save_data` のコードを見てください。このコードを変更する必要はありません。

In [ ]:
"""
このセルでは何も変更しないでください
"""
# トレーニングデータの前処理
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# チェックポイント
ここが最初のチェックポイントです。もし、このノートに戻ることになったり、ノートを再起動することになった場合は、ここからスタートできます。前処理されたデータがディスクに保存されました。

In [ ]:
"""
このセルでは何も変更しないでください
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## ニューラルネットワークの構築
このセクションでは、RNNモジュールとフォワード・バックプロパゲーション関数の実装により、RNNを構築するために必要なコンポーネントを構築します。

### GPUへのアクセスの確認

In [ ]:
"""
このセルでは何も変更しないでください
"""
import torch

# GPUのチェック
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## 入力
まずは、前処理された入力データから始めよう。[TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset)を使用して、データセットに既知のフォーマットを提供する。[DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader)と組み合わせて、バッチ処理、シャッフル、およびその他のデータセットの反復関数を処理する。

TensorDatasetでは、featureとtargetのテンソルを渡してデータを作成します。その後、通常通りDataLoaderを作成します。
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### バッチング
TensorDataset`と`DataLoader`クラスを使って、`words`データを`batch_size`のサイズのチャンクにバッチ処理するために、`batch_data`関数を実装します。

>DataLoaderを使って単語をバッチ処理することができますが、与えられた`sequence_length`に対して適切なサイズと内容の`feature_tensors`と`target_tensors`を作成するのは、あなた次第となります。

例えば、次のような入力があったとします。:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

最初の `feature_tensor` には、これらの値が含まれているはずです。:
```
[1, 2, 3, 4]
```
そして、対応する`target_tensor`は、単に次の "word"/トークン化された単語の値であるべきです。:
```
5
```
これは、2つ目の `feature_tensor` と `target_tensor` で継続して行う必要があります。:
```
[2, 3, 4, 5]  # features
6             # target
```

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    DataLoaderを使ってニューラルネットワークデータをバッチ処理する
    :param words: テレビ番組で使用された単語のID
    :param sequence_length: 各バッチのシーケンス長
    :param batch_size: 各バッチのサイズ（バッチ内のシーケンス数）．
    :return バッチされたデータを持つDataLoader
    """
    # TODO: 関数の実装
    n_batches = len(words)//batch_size
    
    # フルバッチのみ
    words = words[:n_batches*batch_size]
    y_len = len(words) - sequence_length
    x, y = [], []
    for idx in range(0, y_len):
        idx_end = sequence_length + idx
        x_batch = words[idx:idx_end]
        x.append(x_batch)
        batch_y =  words[idx_end]
        y.append(batch_y)

    #Tensor データセットの作成
    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    # トレーニングデータのSHUFFLEを確認
    data_loader = DataLoader(data, shuffle=False, batch_size=batch_size)
    # return a dataloader
    return data_loader

# この関数にはテストがありませんが、独自のテストを作成することをお勧めします。
# 独自のprint文やテストを作成することをお勧めする。


### データローダのテスト 

バッチング関数をテストするためには、このコードを修正する必要がありますが、かなり似ているはずです。

以下では、テスト用のテキストデータを生成し、上記で定義した関数を使ってデータローダを定義しています。そして、dataloader から入力 `sample_x` とターゲット `sample_y` のサンプルバッチを取得しています。

あなたのコードは以下のようなものを返すはずです（データをシャッフルした場合、おそらく異なる順序になるでしょう）:

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### サイズ
sample_x のサイズは `(batch_size, sequence_length)` または (10, 5)とし、sample_y は batch_size (10)の1次元とします。

### 値

また、ターゲットであるsample_yは、順序付けられたtest_textデータの中で、*次の*値であることに気づくはずです。つまり、入力シーケンス `[ 28, 29, 30, 31, 32]` が値 `32` で終わる場合、対応する出力は `33` になるはず。

In [ ]:
# データローダーのテスト

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


---
## ニューラルネットワークの構築
PyTorchの[Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module)を使ってRNNを実装します。GRUまたはLSTMを使用することができます。RNNを完成させるためには、このクラスの以下の関数を実装する必要があります。
 - __init__` - 初期化関数です。
 - `init_hidden` - LSTM/GRUの隠れた状態を初期化する関数です。
 - `forward` - フォワードプロパゲーション関数。
 
初期化関数では、ニューラルネットワークのレイヤーを作成して、クラスに保存します。フォワードプロパゲーション関数は、これらのレイヤーを使ってフォワードプロパゲーションを実行し、出力と隠れた状態を生成します。

**このモデルの出力は、シーケンスが完全に処理された後の、単語スコア**の*最後の*バッチでなければなりません。つまり、入力された一連の単語に対して、次の単語の可能性が最も高い1つの単語のスコアのみを出力したいのです。

### ヒント

1. 完全連結層に渡すlstmの出力を必ず重ねる。`lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`でできる。
2. 最後の完全連結層の出力を次のように整形することで、最後の単語スコアを得ることができます:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# ラストバッチの取得
out = output[:, -1]
```

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        PyTorch RNNモジュールの初期化
        :param vocab_size: ニューラルネットワークの入力次元数（語彙のサイズ）。
        :param output_size: ニューラルネットワークの出力次元数．
        :param embedding_dim: 埋め込みのサイズ（埋め込みを利用する場合）．       
        :param hidden_dim: 隠れ層の出力のサイズ
        :param dropout: LSTM/GRU 層の間に追加するドロップアウト．
        """
        super(RNN, self).__init__()
        # TODO: 関数の実装
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        # クラス変数の設定
        
        # モデルレイヤーの定義
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        ニューラルネットワークの前進伝搬
        :param nn_input: ニューラルネットワークへの入力．
        :param hidden: 隠れた状態        
        :return: ニューラルネットワークの出力と最新の隠れた状態を表す2つのテンソル
        """
        # TODO: 関数の実装   
        batch_size = nn_input.size(0)
        embeds = self.embedding(nn_input)
        lstm_out = self.lstm(embeds, hidden)
        hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # 出力された単語スコアの1バッチと、隠れた状態をreturn 
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        LSTM/GRUの隠れた状態を初期化する
        :param batch_size: 隠れた状態のバッチサイズ
        :return: dims の隠れた状態 (n_layers, batch_size, hidden_dim)
        '''
        # 関数の実装
        
        # 隠れた状態をゼロの重みで初期化し、利用可能であればGPUに移す
        
        return None

"""
これ以下のセルでは何も変更しないでください
"""
tests.test_rnn(RNN, train_on_gpu)

### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [ ]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    
    # perform backpropagation and optimization

    # return the loss over a batch and the hidden state produced by our model
    return None, None

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
これ以下のセルでは何も変更しないでください
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [ ]:
"""
このセルでは何も変更しないでください
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [ ]:
# Data params
# Sequence Length
sequence_length =   # of words in a sequence
# Batch Size
batch_size = 

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [ ]:
# Training parameters
# Number of Epochs
num_epochs = 
# Learning Rate
learning_rate = 

# Model parameters
# Vocab size
vocab_size = 
# Output size
output_size = 
# Embedding Dimension
embedding_dim = 
# Hidden Dimension
hidden_dim = 
# Number of RNN Layers
n_layers = 

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [ ]:
"""
このセルでは何も変更しないでください
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
"""
このセルでは何も変更しないでください
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [ ]:
"""
このセルでは何も変更しないでください
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [ ]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
これ以下のセルでは何も変更しないでください
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.